<a href="https://colab.research.google.com/github/tripsysi/Wine_Classifier/blob/main/DATA7703_Project_Wine_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA7703 Project  - Wine Classifier

In [58]:
####################
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split

## Read in data

In [52]:
redwine = pd.read_csv("https://raw.githubusercontent.com/tripsysi/Wine_Classifier/main/data/winequality-red.csv")
whitewine = pd.read_csv("https://raw.githubusercontent.com/tripsysi/Wine_Classifier/main/data/winequality-white.csv")

Citation : Data from https://archive.ics.uci.edu/ml/datasets/wine+quality

In [53]:
redwine.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [54]:
print(redwine.shape)
print(whitewine.shape)

(1599, 12)
(4898, 12)


## Preprocessing

### Merging Wine tables 

In [55]:
redwine.insert(11, 'Winetype', 0)
whitewine.insert(11, 'Winetype', 1)

In [56]:
print(redwine.shape)
print(whitewine.shape)

(1599, 13)
(4898, 13)


In [57]:
wines = pd.concat([redwine,whitewine])
wines.shape

(6497, 13)

### Normalizing data

In [59]:
X = wines.iloc[: , :-1]
y = wines.iloc[:,-1:]

In [63]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

### Splitting into Train Test sets

## Modelling

### Knn

### Naive Bayes

### Random Forest